In [ ]:
%%capture

import warnings

warnings.filterwarnings("ignore")
import _ct_district_data_prep
# import _ct_district_visuals
import _report_utils
import calitp_data_analysis.magics
import geopandas as gpd
import pandas as pd
from great_tables import GT
from IPython.display import HTML, Image, Markdown, display, display_html
from slugify import slugify
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS

In [ ]:
import google.auth

credentials, project = google.auth.default()

import gcsfs

fs = gcsfs.GCSFileSystem()

In [ ]:
# Comment out and leave this cell right below pandas
district =  '02 - Redding'

In [ ]:
DISTRICT_DIGEST_URL = (
    "https://gtfs-digest--cal-itp-data-analyses.netlify.app/district_"
    f"{slugify(district)}"
)

In [ ]:
# %%capture_parameters
# district, DISTRICT_DIGEST_URL

In [ ]:
# Extract district from district string when doing an sjoin
# between CT districts & routes
district_int = [int(s) for s in district.split() if s.isdigit()][0]

In [ ]:
# Load Datasets
operator_df = _ct_district_data_prep.data_wrangling_operator_profile(district)

operator_gdf = _ct_district_data_prep.data_wrangling_operator_map(
    list(operator_df.portfolio_organization_name.unique())
)

ct_district_gdf = _ct_district_data_prep.load_ct_district(district_int)

transit_route_shs_gdf, transit_route_shs_table = (
    _ct_district_data_prep.final_transit_route_shs_outputs(20, str(district_int))
)
district_gdf = _ct_district_data_prep.load_ct_district(district_int)

In [ ]:
shn_gdf = _ct_district_data_prep.load_buffered_shn_map(50, district_int)

In [ ]:
gtfs_table_df = _ct_district_data_prep.create_gtfs_stats(operator_df)

# District {district}

These are district summaries for [GTFS Digest](https://gtfs-digest--cal-itp-data-analyses.netlify.app/). 

Individual transit operators have their pages at: **[{DISTRICT_DIGEST_URL}]({DISTRICT_DIGEST_URL})**

In [ ]:
district_summary = _report_utils.district_stats(operator_df, "caltrans_district")

In [ ]:
summary_table1 = (
    GT(
        district_summary.drop(columns=["arrivals_per_stop", "trips_per_operator"]).pipe(
            _report_utils.transpose_summary_stats, district_col="caltrans_district"
        )
    )
    .fmt_integer(columns="value")
    .cols_label(index="")
    .tab_header(title=f"District {district} GTFS summary stats")
)

summary_table2 = (
    GT(
        district_summary[
            ["caltrans_district", "arrivals_per_stop", "trips_per_operator"]
        ].pipe(_report_utils.transpose_summary_stats, district_col="caltrans_district")
    )
    .fmt_number("value", decimals=1)
    .cols_label(index="")
)

In [ ]:
display(summary_table1)
display(summary_table2)

## Routes within the District

In [ ]:
m = district_gdf.explore(
    name="District",
    tiles="CartoDB positron",
    style_kwds={"color": "#9DA4A6", "opacity": 0.5},
    height=500,
    width=1000,
)

In [ ]:
m = operator_gdf.explore(
    m=m,
    column="Portfolio Organization Name",
    cmap="RdBu",
    categorical=True,
    legend=True,
    legend_kwds={"width": 200},
)

In [ ]:
display(m)

## Transit Routes on the State Highway Network
**Only transit routes that have 20% or more if its length on one or more State Highway Network routes are included**

In [ ]:
m2 = shn_gdf.loc[shn_gdf.District == district_int].explore(
    height=500,
    width=1000,
    style_kwds={"color": "#9DA4A6", "weight": 6, "opacity": 0.5},
    tiles="CartoDB positron",
    name="shs",
)

In [ ]:
m2 = transit_route_shs_gdf.explore(
    "Percentage of Transit Route on SHN Across All Districts",
    m=m2,
    cmap="Blues",
    legend=True,
)

In [ ]:
display(m2)

In [ ]:
GT(
    transit_route_shs_table.sort_values(
        by=[
            "Portfolio Organization Name",
            "Percentage of Transit Route on SHN Across All Districts",
        ],
        ascending=[True, False],
    )
)

## GTFS Stats by Operator

In [ ]:
string_cols = gtfs_table_df.select_dtypes(include="object").columns.tolist()

In [ ]:
gtfs_table = (
    GT(gtfs_table_df.sort_values("# Trips", ascending=False))
    .fmt_integer(
        columns=[
            c
            for c in gtfs_table_df.columns
            if c not in ["Operator Service Miles", "Avg Arrivals per Stop"]
            and (c not in string_cols)
        ]
    )
    .fmt_number(columns=["Operator Service Miles", "Avg Arrivals per Stop"], decimals=1)
    .data_color(
        columns=["# Trips", "Avg Arrivals per Stop"],
        palette=["white", "green"],
        na_color="lightgray",
    )
    .tab_header(
        title=f"District {district}",
        subtitle="Daily GTFS schedule statistics by operator",
    )
    .cols_align(
        columns=[
            c
            for c in gtfs_table_df.columns
            if c not in ["Organization", "Transit Operator"]
        ],
        align="center",
    )
)

In [ ]:
gtfs_table = _report_utils.great_table_formatting(gtfs_table)
gtfs_table